# Loadbalance Between Two OpenAI Accounts

Simple code example on implementing load balancing with Portkey.

### 1. Create **`two LLM objects`** with **`two different API keys`**

We will also set the *weight* for both the LLMs here - this weight determines how requests are distributed among the LLMs.

In [ ]:
# Install the Portkey SDK
!pip install portkey-ai

# Import necessary modules from Portkey
import portkey
from portkey import Config, LLMOptions

# Create two LLM objects with different account keys and set their weights
llm_a = LLMOptions(provider="openai", api_key="OPENAI_ACCOUNT_1_KEY", weight=0.5)
llm_b = LLMOptions(provider="openai", api_key="OPENAI_ACCOUNT_2_KEY", weight=0.5)

### 2. Implement **`loadbalance`** mode

In [ ]:
# Construct the Porktey client and pass the LLM objects

portkey.config = Config(
    api_key="PORTKEY_API_KEY", mode="loadbalance", llms=[llm_a, llm_b]
)

### 3. Make the **`ChatComplete`** call

**Portkey SDK follows OpenAI SDK signature** - model params **including the model name**, can be passed at the completion call.

Just replce **`openai.ChatCompletion.create`** to **`portkey.ChatCompletion.create`** - **Everything else remains the same**

In [ ]:
response = portkey.ChatCompletions.create(
    model="gpt-4-0613",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "What is the meaning of life, universe and everything?",
        },
    ],
)

print(response)

---

### More Production Features

**⏩ Streaming**

* Just set `stream=True` while making your call

In [ ]:
stream_response = portkey.ChatCompletions.create(
    model="gpt-4-0613",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "What is the meaning of life, universe and everything?",
        },
    ],
    stream=True,
)

for i in stream_response:
    print(i.delta, end="", flush=True)

**🔑 Virtual Keys**

Safeguard your original OpenAI API keys and only expose dynamic virtual keys in the Portkey ecosystem

1. Go to the “Virtual Keys” page on Portkey dashboard and hit the “Add Key” button on the top right corner.
2. Choose your AI provider (OpenAI in this case), assign a unique name to your key, and, and add notes. Your virtual key is ready!

* While constructing your LLM object with Portkey, you can pass the virtual keys instead of API keys with `virtual_key` parameter

In [ ]:
llm_a = LLMOptions(provider="openai", virtual_key="openai-xxxx", weight=0.5)

**👣 Tracing**

Monitor your apps throughout the lifecycle of a request with a singular `trace id`.

* You can set the `trace_id` while constructing your LLM object

In [ ]:
llm_a = LLMOptions(
    provider="openai",
    virtual_key="openai-xxxx",
    weight=0.5,
    trace_id="loadbalance_accounts_test",
)

**📝 Live Feedback**

Portkey’s Feedback API offers a straightforward way to gather weighted feedback from users, allowing you to refine and improve your app.

* Append feedback to any `trace_id` through Portkey's feedback endpoint:

In [ ]:
import requests
import json

headers = {"x-portkey-api-key": "PORTKEY_API_KEY", "Content-Type": "application/json"}
data = {"trace_id": "loadbalance_accounts_test", "value": 1}

send_feedback = requests.post(
    "https://api.portkey.ai/v1/feedback", headers=headers, data=json.dumps(data)
)

**💪 Fallbacks & Retries**

Portkey helps you build resilient apps with automatic fallbacks & retries:

**Fallbacks**: If a primary service or model fails, Portkey will automatically switch to a backup model. <br />
**Retries**: If a request fails, Portkey can be configured to retry the request multiple times.

* Fallbacks is set while construcing your Portkey client by setting `mode="fallback"`
* Retry is set while constructing your LLM object `retry={"attempts": 5}`

In [ ]:
llm_a = LLMOptions(provider="openai", virtual_key="openai-xxx", retry={"attempts": 5})
llm_b = LLMOptions(
    provider="anthropic", virtual_key="anthropic-xxx", retry={"attempts": 5}
)

portkey.config = Config(api_key="PORTKEY_API_KEY", mode="fallback", llms=[llm_a, llm_b])

**🛠️ Integrations**

You can also use Portkey with the OpenAI SDK, Langchain, Llamaindex, and more.

Check out [**Portkey docs**](https://docs.portkey.ai/portkey-docs/integrations) for more info.

* Here's a quick example of using Portkey with the OpenAI SDK:

In [ ]:
import openai

# Set Portkey proxy as the base path
openai.api_base = "https://api.portkey.ai/v1/proxy"

# Set Portkey headers
portkey_headers = {
    "x-portkey-api-key": "PORTKEY_API_KEY",
    "x-portkey-mode": "proxy openai",
}

response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Translate the following English text to French: '{}'",
    headers=portkey_headers,
)

**Join Discord**

Collaborate with industry practitioners building LLM apps and get first-class Portkey support: [**Join here**](https://discord.com/invite/DD7vgKK299)